# Calculus Implementation

Here I attempt to implement a class `Function` to provide utilities for numerical differentiation and integration without the aid of NumPy (just so that I have really think about it, nothing against NumPy!!). 

However, in creating a helper `Array` class to make operations on lists a bit easier, I have followed many definitions used by NumPy, such as the transpose operation being the reversal of dimensions (similarly defined by TensorFlow).

In this notebook I show the result of this implementation and share some notes about what I have learned throughout the process. 

## I. Notes on Calculus

**def** differentiation: 

**def** integration: 

In [1]:
from math_implementations.calculus import Array, Function

## II. Function

`Function` wraps `function` to provide utilities for applying differentiation and integration. A word from a wise friend of mine are two notes about calculus:

1. integrals smooth but derivatives make things spiky
2. integrals are much harder than derivatives

### II.1 Differentiation of functions from R^N to R

The gradient $\nabla$ is an operator that takes in a function and outputs a function. If the input function is defined from $R^N \rightarrow R^M$, then the output function is defined from $R^N \rightarrow R^{NxM}$. The operator computes the partial derivative for each dimension in $R^N$, returning a `vector` of **partial derivatives** (in multiple dimensions each point $P$ has a slope in every direction defined in $R^N$). The partial derivative of dimension $N_i$ is the directional derivative in the direction given by the vector $[\ldots, 1, \ldots]$ `1` in the `ith` dimension, `0` elsewhere, namely, it is the slope along a single axis.

#### II.1.1 The gradient as the direction of steepest ascent
In addition, this gradient, evaluated at point $P$, points in the direction of the steepest ascent at point $P$. Let the gradient of $f$ at point P, be the vector $\vec{v}$

$$\nabla f(P) = \vec{v}$$

With the gradient of $f$ being the vector $\vec{v}$, the **directional derivative** of $f$ in the direction of the unit-vector (we only care about the direction not the magnitude) $\vec{u}$ at point $P$ is

$$\vec{v} \cdot \vec{u}$$

We want to find the direction of the steepest ascent, in other words the direction $\vec{u}$ which maximizes $\vec{v}$. To do this, we use an alternate definition of the dot product between two vectors

$$||\vec{v}|| \times ||\vec{u}|| \times \cos{\theta}$$

where $\theta$ is the angle between the two vectors. Now, $\vec{u}$ is a **unit vector** and so its magnitude is `1`. Therefore, maximizing the above equation is equivalent to maximizing $\cos{\theta}$. $\cos{\theta}$ is maximized at `0` meaning that the direction where $||\vec{v}|| \times ||\vec{u}|| \times \cos{\theta}$ is maximized is when $\vec{u}$ points in the **same** direction as $\vec{v}$, i.e. it points in the same direction as the gradient.

This concept is not important for the calculus sub-package or this notebook, but it is import for gradient descent, a topic for computation optimization, which I hope to also implement. And I just found it interesting enough to try and actually understand it.

$$f(x, y): R^2 \rightarrow R = x^2 + y^2$$ 

In [2]:
func = lambda x, y: (x ** 2) + (y ** 2)
f = Function(function_def=func, num_inputs=2, output_dims=None)  # a function from R^N -> R is scalar-valued so its output has not dimensionality

#### II.1.2 Notice how the shape of the output changes as we increase the number of derivates taken

In [3]:
print(f"f    : {f.output_dims}")  # None
print(f"f'   : {f.differentiate.output_dims}")  # [df/dx, df/dy]
print(f"f''  : {f.differentiate.differentiate.output_dims}")  # [[d2f/dx2, d2f/dxy], [d2f/dy2, d2f/dyx]]
print(f"f''' : {f.differentiate.differentiate.differentiate.output_dims}")  # [[[d3f/dx3, d3f/dx2y], [d3f/dx2y, d3f/dxy2]], [[d3f/dy3, d3f/dy2x], [d3f/dy2x, d3f/dyx2]]]

f    : None
f'   : (2,)
f''  : (2, 2)
f''' : (2, 2, 2)


#### II.1.3 Computational derivative

$$ f(x_{0:N}): R^N \rightarrow R$$

\begin{align}
f_{xi}(x_{0:N}) &= \frac{f(x_0, \cdots, x_i + \epsilon, \cdots, x_N)) - f(x_0, \cdots, x_N)}{\epsilon} && \text{jitter right} \\
&= \frac{f(x_0, \cdots, x_N) - f(x_0, \cdots, x_i - \epsilon, \cdots, x_N))}{\epsilon} && \text{jitter left} \\
&= \frac{f(x_0, \cdots, x_i - \epsilon, \cdots, x_N)) - f(x_0, \cdots, x_i + \epsilon, \cdots, x_N)}{2\epsilon} && \text{jitter both left and right} \\
\end{align}

The definition of `differentiable` is garnered from the first two methods. If $f_{xi}(x_{0:N})$ from the right and from the left are approximately equal at point $P$ then we consider it differentiable at point $P$. 

Out of the three methods above, the last one, jittering both to left and to the right of point $P$, is considered the least biased. However, in this sub-package we use the first method, for computational ease.

In [4]:
f.differentiate(2, 2)  # the gradient is the vector [2x, 2y]

[4.000099999998952, 4.000099999998952]

In [5]:
f.differentiate.differentiate(2, 2)  # the second derivative is the vector [[2x, 0], [2y, 0]]

[[2.000000165480742, 1.7763568394002505e-07], [1.7763568394002505e-07, 2.000000165480742]]

### II.2 Integration of functions from R^N to R

The integral $\int$ is an operator that takes in a function and outputs a function. If the input function is defined from $R^N \rightarrow R$, then the output function is also defined from $R^N \rightarrow R$. The operator computes the space (area, volume, hyper-volume) under the curve defined by $f$. In the case of explicit integration, like here, the integral is only defined over a **specific interval**. 

#### II.2.1 Integrating over an interval

In the case of integrating a function $f(x): R \rightarrow R$, the interval would be an interval on the x-axis 

$$\int_a^b f(x) dx$$

In the case of integrating a function $f(x, y): R^2 \rightarrow R$, the interval would be a rectangle over the x- and y-axes

$$\int_c^d\int_a^b f(x, y) dxdy$$

and so on as we increase dimensions. However, this gets a bit tricky, if we don't have $f(x, y)$, but instead we have a gradient $[\frac{df}{dx}, \frac{df}{dy}]$, a vector of partial derivatives that we want to integrate to get back $f(x, y)$.

#### II.2.2 Integral or Antiderivative?

This has thrown me a bit for a loop, but playing fast-and-loose the antiderivative can be thought of as the implicit integral.

- implicit integration: $\int f'(x) dx = f(x) + C$ 
- explicit integration: $\int_a^b f'(x) dx = f(x) + C$ 

#### II.2.3 How to integrate the gradient

Let's define $f(x, y) = x^2 + 2xy + y^2$, therefore $\nabla f(x, y) = [2x + 2y, 2x + 2y]$. Let's attempt to derive $f(x, y)$ from $f'(x, y)$.


From the **gradient theorem**, or the fundamental theorem of calculus for line integrals, we obtain the tidbit that line integrals are path independent throught gradient fields! This means that 

\begin{align}
f(x, y) &= \int \frac{df}{dx} dx = \int \frac{df}{dy} && \text{let's choose the integral of df/dx just because}  \\
f(x, y) &= x^2 + 2xy + C(y) && \text{where C(y) is some constant function of y} \\
C(y) &= f(x, y) - x^2 - 2xy && \text{isolate C(y)} \\
\frac{d}{dy} C(y) &= \frac{d}{dy}f(x, y) - \frac{d}{dy}(x^2 + 2xy) && \text{we can do derivatives pretty well}\\
C'(y) &= (2x + 2y) - 2x && \text{we know that df/dy = 2x + 2y !!} \\
\int C'(y) dy &= \int 2y dy && \text{integrate with respect to y to get back to C(y)} \\
C(y) &= y^2 + C && \text{nice} \\
f(x, y) &=x^2 + 2xy + y^2 + C && \text{plug C(y) back in and we got it!} \\
\end{align}

So that's how we can integrate the gradient to get back to the original function.

If you stare at this long enough, you actually start to see a trick of the form 

$$ f(x, y) = \int \frac{df}{dx} dx + \int \text{[each element of df/dy that does not have an x in it]} dy$$

#### II.2.4 Notice how the shape of the output doesn't change as we increase the number of integrations taken

In [6]:
print(f"f    : {f.output_dims}")  # None
print(f"f'   : {f.integrate.output_dims}")  # None
print(f"f''  : {f.integrate.integrate.output_dims}")  # None
print(f"f''' : {f.integrate.integrate.integrate.output_dims}")  # None

f    : None
f'   : None
f''  : None
f''' : None


#### II.2.5 Computational integration

It's all about Reimann sums for now until we get smarter.

$$\int_a^b f(x) = \lim_{\Delta x \rightarrow 0} \sum_{i=a}^b f(i) * \Delta x$$
$$\iint_a^b f(x, y) = \lim_{\Delta x,y \rightarrow 0} \sum_{i=a}^b\sum_{j=a}^b f(i, j) * \Delta x * \Delta y$$
$$\cdots$$

This get **really** slow even integrating functions from $R^2 \rightarrow R$ and so we have developed smarter tools for approximating integrals like MCMC.

So, this library is not good at integrating functions larger than $R \rightarrow R$, but it does implement integrating the gradient of functions from $R^N \rightarrow R$ which is cool and not slow :)

### II.3 Demo

In [7]:
def demo_func(x, y, z): 
    return [
        (xi ** 2) + (yi ** 3) + (zi ** 4) + (xi / ((yi + 1) ** 2)) + ((yi ** 2) * (zi ** 0.5))
        for xi, yi, zi in zip(x, y, z)
    ]
demo_f = Function(demo_func, num_inputs=3, output_dims=None)

print(demo_f(2, 2, 2))
print(demo_f.differentiate.integrate(0, 2))  # integrate over the 2 x 2 x 2 cube

33.879076471714605
33.88448052494364


## III. Array

`Array` wraps `list` to provide methods for element-wise operations and other nice things.

In [8]:
a = [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12]],

     [[13, 14, 15, 16],
      [17, 18, 19, 20],
      [21, 22, 23, 24]]]  # a 3 dimensional matrix

### III.1 Shape and transposition

In [9]:
A = Array(a)
print(A.shape)
print(A.T.shape)

(2, 3, 4)
(4, 3, 2)


### III.2 Element-wise operations

In [10]:
print(f"A\n{A}", "\n\n")
print(f"A + 1\n{A + 1}", "\n\n")
print(f"A * 10\n{A * 10}", "\n\n")
print(f"A > 12\n{A > 12}", "\n\n")
print(f"[1, 2, 3, 4] + [1, 2, 3, 4]\n{Array([1, 2, 3, 4]) + Array([1, 2, 3, 4])}","\n\n")

A
[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]] 


A + 1
[[[2, 3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13]], [[14, 15, 16, 17], [18, 19, 20, 21], [22, 23, 24, 25]]] 


A * 10
[[[10, 20, 30, 40], [50, 60, 70, 80], [90, 100, 110, 120]], [[130, 140, 150, 160], [170, 180, 190, 200], [210, 220, 230, 240]]] 


A > 12
[[[False, False, False, False], [False, False, False, False], [False, False, False, False]], [[True, True, True, True], [True, True, True, True], [True, True, True, True]]] 


[1, 2, 3, 4] + [1, 2, 3, 4]
[2, 4, 6, 8] 




### III.3 Indexing

In [11]:
print(f"A[:, 0, 0]\n{A[:, 0, 0]}", "\n\n")
print(f"A[:, :, 0]\n{A[:, :, 0]}", "\n\n")
print(f"A[0, :, -1]\n{A[0, :, -1]}", "\n\n")

A[:, 0, 0]
[1, 13] 


A[:, :, 0]
[[1, 2, 3, 4], [13, 14, 15, 16]] 


A[0, :, -1]
[4, 8, 12] 


